In [ ]:
!pip install transformers
!pip install einops
!pip install gdown
!pip install pandas

!pip install bitsandbytes
!pip install torch
!pip install loralib
!pip install peft
!pip install accelerate
!pip install datasets
!pip install optimum auto-gptq
!pip install trl
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyli

In [ ]:
from transformers import pipeline, Conversation, AutoTokenizer, AutoModelForCausalLM, TextStreamer
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from trl import SFTTrainer
import transformers
import bitsandbytes as bnb
from huggingface_hub import notebook_login
import torch
import gdown
import pandas as pd
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AwqConfig,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Dataset

In [ ]:
gdown.download(id="1C8ABLukXRuEEXaFEWKH2TP8Id0LbQXnrs5BUqYbHEgE", output="Dataset.xlsx")

Downloading...
From (original): https://drive.google.com/uc?id=1C8ABLukXRuEEXaFEWKH2TP8Id0LbQXnrs5BUqYbHEgE
From (redirected): https://docs.google.com/spreadsheets/d/1C8ABLukXRuEEXaFEWKH2TP8Id0LbQXnrs5BUqYbHEgE/export?format=xlsx
To: /content/Dataset.xlsx
46.0kB [00:00, 231kB/s]


'Dataset.xlsx'

In [ ]:
read_file = pd.read_excel("Dataset.xlsx")
read_file.to_csv("Dataset.csv",
                  index = None,
                  header=True)
dataset = load_dataset("csv", data_files="Dataset.csv")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Label', 'Prompt', 'Completion'],
        num_rows: 49
    })
})

# Dataset 2

In [ ]:
# gdown.download(id="1BrSr_QmqTbleYjRAhJTWwUEsuzZZy1X2", output="Dataset 2.txt")

In [ ]:
# dataset2 = load_dataset("text", data_files="Dataset 2.txt")
# dataset2

# Dataset 3

In [ ]:
# gdown.download(id="1qec4rMi0TJCbQ-9O34vDTipedtDmmJCE", output="Dataset 3.txt")

In [ ]:
# dataset3 = load_dataset("text", data_files="Dataset 3.txt")
# dataset3

# Merge Datasets

In [ ]:
# from datasets import concatenate_datasets

# def add_columns(dataset):
#     # Adding dummy values for 'Labels', 'Prompt', and 'Completion'
#     dataset = dataset.map(lambda example: {"Labels": None, "Prompt": None, "Completion": example["text"]})
#     return dataset.remove_columns(["text"])

# dataset2 = add_columns(dataset2)
# dataset3 = add_columns(dataset3)

In [ ]:
# print(dataset3['train'])

In [ ]:
# merged_dataset = concatenate_datasets([dataset['train'], dataset2['train'], dataset3['train']])

In [ ]:
# print(merged_dataset)
# print(merged_dataset[113])

# Init Model

In [ ]:
device = "cuda:0"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map=device)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# Prepare Training

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [ ]:
# target_modules = []

# for i in range(32):
#     target_modules.extend([
#         f"model.layers.{i}.self_attn.q_proj",
#         f"model.layers.{i}.self_attn.k_proj",
#         f"model.layers.{i}.self_attn.v_proj",
#         f"model.layers.{i}.self_attn.o_proj",
#         f"model.layers.{i}.mlp.gate_proj",
#         f"model.layers.{i}.mlp.up_proj",
#         f"model.layers.{i}.mlp.down_proj",
#     ])

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 41943040 || all params: 8072204288 || trainables%: 0.5195983464188562


# Making Prompt

In [ ]:
dataset["train"]["Label"]

[None,
 None,
 None,
 'visual',
 'kinestetik',
 'auditori',
 'verbal',
 None,
 None,
 None,
 'kinestetik',
 None,
 None,
 'auditori',
 None,
 'visual',
 None,
 None,
 None,
 None,
 'verbal',
 None,
 None,
 None,
 None,
 None,
 'visual',
 'kinestetik',
 'auditori',
 'verbal',
 'visual',
 'kinestetik',
 'auditori',
 'verbal',
 None,
 None,
 None,
 None,
 'Gaya Belajar',
 'Psikoedukasi ',
 None,
 'Model Pembelajaran',
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
def generate_prompt(data_point):
  return f"""
user: {data_point["Prompt"]}
assistant: {data_point["Completion"]}
""".strip()

def generate_prompt_label(data_point):
  return f"""
domain: {data_point["Label"]}
user: {data_point["Prompt"]}
assistant: {data_point["Completion"]}
""".strip()

def generate_completion_only(data_point):
  return f"""
domain: {data_point["Label"]}
user: {data_point["Prompt"]}
assistant: {data_point["Completion"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

def generate_and_tokenize_prompt_label(data_point):
  full_prompt = generate_prompt_label(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

def generate_and_tokenize_completion_only(data_point):
  full_prompt = generate_prompt_label(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

def generate_and_tokenize(data_point):
    # Check if the 'Label' is NaN
    if pd.isna(data_point["Label"]):
        return generate_and_tokenize_prompt(data_point)
    elif pd.isna(data_point["Prompt"]):
        return generate_and_tokenize_completion_only(data_point)
    else:
        return generate_and_tokenize_prompt_label(data_point)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
# dataset = merged_dataset.shuffle().map(generate_and_tokenize)
dataset = dataset['train'].shuffle().map(generate_and_tokenize)

dataset

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

Dataset({
    features: ['Label', 'Prompt', 'Completion', 'input_ids', 'attention_mask'],
    num_rows: 49
})

In [ ]:
print(dataset['Completion'])

['Anak-anak dengan kecerdasan verbal linguistik sangat baik dalam menggunakan kata-kata, baik dalam bentuk tulisan maupun lisan. Mereka biasanya memiliki keterampilan tinggi dalam menulis, berbicara, menghafal informasi, dan menikmati membaca. Berikut adalah beberapa metode yang dapat dilakukan untuk mengoptimalkan potensi mereka:\n\n- Membaca Buku Secara Aktif: Anak-anak dapat mengembangkan keterampilan pemahaman membaca, analisis kritis, dan memperluas pengetahuan mereka tentang berbagai topik.\n- Diskusi Kelompok atau Debat: Membantu anak untuk mengartikulasikan pikiran mereka secara verbal, mendengarkan argumen orang lain, dan merespons secara kritis.\n- Menulis Cerita atau Esai: Memungkinkan anak untuk mengasah keterampilan menulis mereka, mengorganisir ide-ide secara logis, dan mengungkapkan pemikiran dan perasaan mereka dengan jelas.\n- Pembacaan dan Analisis Teks: Belajar dengan membaca buku, artikel, atau cerita yang sesuai dengan minat mereka, dan kemudian menganalisis konten

# Finetuning

In [ ]:
# for param in model.parameters():
#     param.requires_grad = True

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      save_total_limit=3,
      logging_steps=1,
      output_dir="./results",
      optim="paged_adamw_8bit",
      lr_scheduler_type="constant",
      warmup_ratio=0.05,
)

# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=dataset,
#     args=training_args,
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
# )

data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    packing=False,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1847: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/u

Step,Training Loss
1,1.443600
2,1.593100
3,1.530100
4,1.175200
5,1.231500
6,1.036600
7,1.250800
8,1.559000
9,1.061800
10,0.960100


TrainOutput(global_step=12, training_loss=1.2671514054139454, metrics={'train_runtime': 83.091, 'train_samples_per_second': 0.59, 'train_steps_per_second': 0.144, 'total_flos': 1394298903527424.0, 'train_loss': 1.2671514054139454, 'epoch': 0.98})

# Save Trained Model

In [ ]:
repo_name = "Vermillion8631/llama-3-teachtechai"
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vermillion8631/llama-3-teachtechai/commit/2ef9d0da3dd1b770c1467facb1b779c3ec3eff33', commit_message='Upload tokenizer', commit_description='', oid='2ef9d0da3dd1b770c1467facb1b779c3ec3eff33', pr_url=None, pr_revision=None, pr_num=None)

# Merge Adapters

In [ ]:
from peft import PeftConfig
config = PeftConfig.from_pretrained(repo_name)

adapter_config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, torch_dtype=torch.float16, return_dict=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, repo_name)

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path,
                                                       padding_side='left',
                                                       add_eos_token=True
                                                       )
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
merged_model = model.merge_and_unload()

In [ ]:
merged_repo = "Vermillion8631/llama-3-teachtechai-merged"

merged_model.push_to_hub(merged_repo)
tokenizer.push_to_hub(merged_repo)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vermillion8631/llama-3-teachtechai-merged/commit/611005f20f96e7e1034d8481464a0e78ba734aed', commit_message='Upload tokenizer', commit_description='', oid='611005f20f96e7e1034d8481464a0e78ba734aed', pr_url=None, pr_revision=None, pr_num=None)

# Quantization

In [ ]:
torch.cuda.empty_cache()

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(repo_name,
                                                       padding_side='left',
                                                       add_eos_token=True
                                                       )
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
data_strings = [f"{item['Prompt']} {item['Completion']}" for item in dataset]
data_strings

['Apa metode pembelajaran yang tepat untuk mengajar siswa dengan tipe kecerdasan verbal atau linguistik? Anak-anak dengan kecerdasan verbal linguistik sangat baik dalam menggunakan kata-kata, baik dalam bentuk tulisan maupun lisan. Mereka biasanya memiliki keterampilan tinggi dalam menulis, berbicara, menghafal informasi, dan menikmati membaca. Berikut adalah beberapa metode yang dapat dilakukan untuk mengoptimalkan potensi mereka:\n\n- Membaca Buku Secara Aktif: Anak-anak dapat mengembangkan keterampilan pemahaman membaca, analisis kritis, dan memperluas pengetahuan mereka tentang berbagai topik.\n- Diskusi Kelompok atau Debat: Membantu anak untuk mengartikulasikan pikiran mereka secara verbal, mendengarkan argumen orang lain, dan merespons secara kritis.\n- Menulis Cerita atau Esai: Memungkinkan anak untuk mengasah keterampilan menulis mereka, mengorganisir ide-ide secara logis, dan mengungkapkan pemikiran dan perasaan mereka dengan jelas.\n- Pembacaan dan Analisis Teks: Belajar deng

In [ ]:
quantization_config = transformers.GPTQConfig(bits=4,
                                              group_size=128,
                                              desc_act=False,
                                              dataset=data_strings,
                                              tokenizer=tokenizer)

In [ ]:
q_model = AutoModelForCausalLM.from_pretrained(merged_repo, quantization_config=quantization_config, torch_dtype=torch.float16, device_map="auto")

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


In [ ]:
gptq_repo = "Vermillion8631/llama-3-teachtechai-gptq"

q_model.push_to_hub(gptq_repo)
tokenizer.push_to_hub(gptq_repo)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vermillion8631/llama-3-teachtechai-gptq/commit/c845b5b3e3a5dc8a939e41f86bcc2775f1cf1c28', commit_message='Upload tokenizer', commit_description='', oid='c845b5b3e3a5dc8a939e41f86bcc2775f1cf1c28', pr_url=None, pr_revision=None, pr_num=None)